In [1]:
from langchain.agents import Tool
from SPARQLWrapper import SPARQLWrapper, JSON

def query_virtuoso_kg(sparql_query: str) -> str:
    sparql = SPARQLWrapper("http://192.168.133.39:8890/sparql")  # 確保這是正確的 SPARQL 端點 URL
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    # 格式化查詢結果，將它轉換為易讀的字串格式
    formatted_results = "\n".join(
        [f"Result {i+1}: {result}" for i, result in enumerate(results["results"]["bindings"])]
    )
    return formatted_results

# 查詢範例，加入命名空間定義
sparql_query = """
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?occupation WHERE {
  dbr:蔣中正 dbo:occupation ?occupation
}
LIMIT 1
"""

# 註冊為工具
sparql_tool = Tool(
    name="VirtuosoKGQuery",
    func=query_virtuoso_kg,
    description="查詢本地知識圖譜的 SPARQL 工具，使用 RDF 知識"
)



In [2]:
from langchain_openai import ChatOpenAI
import os

# 設置 LLM 模型
#API#1
#os.environ["GROQ_API_KEY"] = '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'
#API#2
#s.environ["GROQ_API_KEY"] = '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'
#API#3
os.environ["GROQ_API_KEY"] = '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'
#API#4
#os.environ["GROQ_API_KEY"] = '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'
#API#5
#os.environ["GROQ_API_KEY"] = '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'

llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ['GROQ_API_KEY'],

    model_name="llama-3.3-70b-versatile",       

    temperature=0.0,
  #  max_tokens=1000,
)

In [16]:
import langchain
# 設定 debug 模式為 False
#langchain.debug = True
langchain.debug = False

print(langchain.__version__)

0.3.14


In [5]:

# 初始化 Agent，將 SPARQL 查詢工具傳遞給 Agent
agent = initialize_agent(
    tools=[sparql_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 使用 agent 查詢本地知識圖譜
response = agent.run("請查詢 '蔣中正' 的 label")
print(response)





> Entering new AgentExecutor chain...
Thought: 我需要使用 SPARQL 查詢工具來查詢 '蔣中正' 的 label。首先，我需要知道 '蔣中正' 的 URI，然後才能查詢他的 label。

Action: SPARQL 查詢工具
Action Input: PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT ?label WHERE { <http://zh.dbpedia.org/resource/蔣中正> rdfs:label ?label }


ConnectionError: HTTPConnectionPool(host='l192.168.133.39', port=8890): Max retries exceeded with url: /sparql?query=PREFIX+rdf%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23%3E+PREFIX+rdfs%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E+SELECT+%3Flabel+WHERE+%7B+%3Chttp%3A%2F%2Fzh.dbpedia.org%2Fresource%2F%E8%94%A3%E4%B8%AD%E6%AD%A3%3E+rdfs%3Alabel+%3Flabel+%7D%0A (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000002D19B402350>: Failed to resolve 'l192.168.133.39' ([Errno 11001] getaddrinfo failed)"))